In [1]:
# import libraries

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import os
# Plot style
plt.style.use('fivethirtyeight')

In [2]:
# Check matplotlib version
matplotlib.__version__

'3.5.2'

In [3]:
# This option ensures the charts you create are displayed in the notebook without the need to "call" them specifically.
%matplotlib inline

In [4]:
# Turn project folder path into a string
path = r'/Users/peterreadman/Desktop/Python Projects/PortfolioProjects/HealthcareSpending/'

In [5]:
# import person data

df = pd.read_csv(os.path.join(path,'02 Data','02b Prepared Data', 'asec_22_prep.csv'))

In [6]:
df.head()

,Unnamed: 0,Person_ID,Household_ID,Per_ID_WithinHH,PerH_ID,Age,Sex,Emp_Type,Medicare_Premium,Premium,...,Tot_Per_Income,Income_Bracket,Region,State_Name,MOOP_tot,Out_of_Pocket,Pct_Inc_oop,Current_Coverage,Insurance_Type,Employer_Paid
0,0,2022093493100051201101,1,41,1_41,42,Male,private,0,3840,...,44220,18,1,Maine,3840,3840,8.7,yes,Marketplace,niu
1,1,2100059324402431401101,5,41,5_41,62,Male,SE,0,200,...,32702,14,1,Maine,568,568,1.7,yes,Employment,niu
2,2,2100059324402431401102,5,42,5_42,64,Female,gov_state,0,1200,...,50050,21,1,Maine,1300,1300,2.6,yes,Employment,Some
3,3,1410320300869091411101,6,41,6_41,26,Male,private,0,0,...,40001,17,1,Maine,100,100,0.2,no,Unknown,niu
4,4,1410320300869091411102,6,42,6_42,24,Female,private,0,1000,...,20000,9,1,Maine,1400,1400,7.0,no,Indian_HS,niu


In [7]:
df['Emp_Type'].value_counts()

niu             78058
private         56054
SE               4825
gov_loc          4742
gov_state        3759
SE_inc           2891
gov_fed          2170
never_worked      191
no_pay             42
Name: Emp_Type, dtype: int64

# Create a new dataframe itemizing Out of Pocket Costs by 'type'

In [8]:
columns = [#'Person_ID',
            'Household_ID',
            'Per_ID_WithinHH',
            #'Age',
            #'Sex',
            #'Emp_Type',
            'Medicare_Premium',
            'Premium',
            'Non_Premium_OutOfPocket',
            'OverTheCounter_Exp']
            #'Tot_Per_Income',
            #'Income_Bracket',
            #'Region',
            #'State_Name',
            #'MOOP_tot',
            #'Out_of_Pocket',
            #'Pct_Inc_oop']

In [9]:
df_oop = df[columns].copy()

In [10]:
df_oop.head()

,Household_ID,Per_ID_WithinHH,Medicare_Premium,Premium,Non_Premium_OutOfPocket,OverTheCounter_Exp
0,1,41,0,3840,0,0
1,5,41,0,200,320,48
2,5,42,0,1200,0,100
3,6,41,0,0,0,100
4,6,42,0,1000,300,100


In [11]:
df_oop.columns

Index(['Household_ID', 'Per_ID_WithinHH', 'Medicare_Premium', 'Premium',
       'Non_Premium_OutOfPocket', 'OverTheCounter_Exp'],
      dtype='object')

In [12]:
# Assuming your DataFrame is called 'df_oop'

# Create a list to store the transformed rows
new_rows = []

# Initialize a counter for the Item_ID
item_counter = 1

# Iterate over each row in the DataFrame
for _, row in df_oop.iterrows():
    # Check if the medical expenses are greater than 0
    if row['Medicare_Premium'] > 0:
        new_row = row.copy()
        new_row['Type'] = 'Medicare_Premium'
        new_row['Item_ID'] = item_counter
        item_counter += 1
        new_row['Premium'] = 0
        new_row['Non_Premium_OutOfPocket'] = 0
        new_row['OverTheCounter_Exp'] = 0
        new_rows.append(new_row)
    if row['Premium'] > 0:
        new_row = row.copy()
        new_row['Type'] = 'Premium'
        new_row['Item_ID'] = item_counter
        item_counter += 1
        new_row['Medicare_Premium'] = 0
        new_row['Non_Premium_OutOfPocket'] = 0
        new_row['OverTheCounter_Exp'] = 0
        new_rows.append(new_row)
    if row['Non_Premium_OutOfPocket'] > 0:
        new_row = row.copy()
        new_row['Type'] = 'Non_Premium_OutOfPocket'
        new_row['Item_ID'] = item_counter
        item_counter += 1
        new_row['Medicare_Premium'] = 0
        new_row['Premium'] = 0
        new_row['OverTheCounter_Exp'] = 0
        new_rows.append(new_row)
    if row['OverTheCounter_Exp'] > 0:
        new_row = row.copy()
        new_row['Type'] = 'OverTheCounter_Exp'
        new_row['Item_ID'] = item_counter
        item_counter += 1
        new_row['Medicare_Premium'] = 0
        new_row['Premium'] = 0
        new_row['Non_Premium_OutOfPocket'] = 0
        new_rows.append(new_row)

# Create a new DataFrame with the transformed rows
new_df = pd.DataFrame(new_rows)

# Reset the index of the new DataFrame
new_df.reset_index(drop=True, inplace=True)

In [13]:
new_df.head(5)

,Household_ID,Per_ID_WithinHH,Medicare_Premium,Premium,Non_Premium_OutOfPocket,OverTheCounter_Exp,Type,Item_ID
0,1,41,0,3840,0,0,Premium,1
1,5,41,0,200,0,0,Premium,2
2,5,41,0,0,320,0,Non_Premium_OutOfPocket,3
3,5,41,0,0,0,48,OverTheCounter_Exp,4
4,5,42,0,1200,0,0,Premium,5


In [14]:
# Create a mask to identify rows with more than one non-null value
multiple_values_mask = (new_df[['Medicare_Premium', 'Premium', 'Non_Premium_OutOfPocket', 'OverTheCounter_Exp']].notnull().sum(axis=1) > 1)

# Count the number of rows with more than one non-null value
count = multiple_values_mask.sum()

# Display the count
print("Number of rows with values in more than one column:", count)

Number of rows with values in more than one column: 278287


### Create new column for combined HH and Person ID 'PerH_ID'

In [15]:
# Convert numerical columns to strings
new_df['Household_ID'] = new_df['Household_ID'].astype(str)
new_df['Per_ID_WithinHH'] = new_df['Per_ID_WithinHH'].astype(str)

# Create the 'PerH_ID' column by concatenating 'Household_ID' and 'Per_ID_WithinHH'
new_df['PerH_ID'] = new_df['Household_ID'] + '_' + new_df['Per_ID_WithinHH']

In [55]:
new_df.head()

,Household_ID,Per_ID_WithinHH,Medicare_Premium,Premium,Non_Premium_OutOfPocket,OverTheCounter_Exp,Type,Item_ID,PerH_ID
0,1,41,0,3840,0,0,Premium,1,1_41
1,5,41,0,200,0,0,Premium,2,5_41
2,5,41,0,0,320,0,Non_Premium_OutOfPocket,3,5_41
3,5,41,0,0,0,48,OverTheCounter_Exp,4,5_41
4,5,42,0,1200,0,0,Premium,5,5_42


### Export OOP Cost Type dataframe as csv for analysis

In [56]:
# Export to csv
new_df.to_csv(os.path.join(path,'02 Data','02b Prepared Data', 'asec_22_OOP_costType.csv'))